<a href="https://colab.research.google.com/github/tasinfrancesco/Practical_ML_PSL/blob/main/explicit_encoding_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
df = pd.read_csv("train.csv")


In [ ]:
# print(df.describe())
print(df.head(5))
# dt = pd.read_csv("test.csv")
# print(dt[["UTC" not in cd for cd in dt["creation_date"]]])
type(df["creation_date"][0])
print(strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z"))

print(strptime(df["creation_date"][1], "%Y-%m-%d %H:%M:%S.%f %Z"))
print(df["creation_date"][0])
print(df["creation_date"][1])

def timeinfo(timestr):
  time_struct = strptime(df["creation_date"][0], "%Y-%m-%d %H:%M:%S.%f %Z")

  #is it a weekend
  if time_struct.tm_wday >= 5:
    weekend = 1
  else:
    weekend = 0

  #is it a vacation



Ideas for feature engineering:

  - local comment frequency (estimated from change in post_id over time window)
  - is it a vacation
  - is it a weekend

post

  - post length
  - positivity of content?
  - politeness?

BERT:
  - encode in some way politeness? (there's a stanford thingy)
  - CodeBERT vs BERT?
    - Padding and truncation (how long?)
  - hide vectors of code/cut it out for text processing?
  - clarity as:
    - readability
    - sentence/word length
    - coding depth (nesting)
    - number of questions
    - presence of errors
    - number of topics mentioned ("I tried...")

User ID?

  - is it better to predict for a given user (average) or for single posts?

text vectorisation:

analysis:

  - logistic regression (not great given collinearity)
  - random trees?

predict score, then guess target (not target directly)

Whether the output includes both the code and the error? (not applicable to everything though)


In [ ]:
# !pip install sentencepiece
# !pip install protobuf==3.20.0
from collections import defaultdict, Counter
import json
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")
model = AutoModelForTokenClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github")

model.safetensors:   0%|          | 0.00/596M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs_1_to_10 = []
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]
for t in df["text"][0:10]:
  tok = tokenizer.tokenize(t)
  ids = tokenizer.convert_tokens_to_ids(tok)
  ids_special_tokens = cls + ids + sep

  decoded_str = tokenizer.decode(ids_special_tokens)

  print("start:               ", t)
  print("tokenize:            ", tok)
  print("convert_tokens_to_ids", ids)
  print("add special tokens:  ", ids_special_tokens)
  print("================")
  print("decode:              ", decoded_str)
  print("\n\n")




start:                Sincerely appreciate your taking a look. I never have to query across partitions. Disk space is not an issue. The volume of data definitely justifies the partitioning. Separate tables and or databases are not the option. Still my question remains unanswered :-( Many thanks in advance.
tokenize:             ['Sincerely', 'appreciate', 'your', 'taking', 'a', 'look', '.', 'I', 'never', 'have', 'to', 'query', 'across', 'partitions', '.', 'Disk', 'space', 'is', 'not', 'an', 'issue', '.', 'The', 'volume', 'of', 'data', 'definitely', 'justifies', 'the', 'partitioning', '.', 'Separate', 'tables', 'and', 'or', 'databases', 'are', 'not', 'the', 'option', '.', 'Still', 'my', 'question', 'remains', 'unanswered', ':', '-', '(', 'Many', 'thanks', 'in', 'advance', '.']
convert_tokens_to_ids [52297, 5429, 1881, 4670, 69, 2076, 18, 45, 3071, 1851, 1790, 2400, 4036, 12207, 18, 17326, 3257, 1804, 1859, 1794, 2456, 18, 1911, 7817, 1815, 1943, 5480, 57754, 1783, 14744, 18, 21624, 3133

In [30]:
# !pip install evaluate
#Imports
from torch.optim import AdamW
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import evaluate
from transformers import Trainer, AutoModelForTokenClassification, AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup
from datasets import DatasetDict, load_dataset, Dataset
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import torch
import datasets


In [27]:
!pip show torch

Name: torch
Version: 2.9.0+cpu
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org
Author: 
Author-email: PyTorch Team <packages@pytorch.org>
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, setuptools, sympy, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision


In [11]:
#DEFINE ROCAUC optimizer

_DESCRIPTION = """
This metric computes the area under the curve (AUC) for the Receiver Operating Characteristic Curve (ROC). The return values represent how well the model used is predicting the correct classes, based on the input data. A score of `0.5` means that the model is predicting exactly at chance, i.e. the model's predictions are correct at the same rate as if the predictions were being decided by the flip of a fair coin or the roll of a fair die. A score above `0.5` indicates that the model is doing better than chance, while a score below `0.5` indicates that the model is doing worse than chance.
This metric has three separate use cases:
    - binary: The case in which there are only two different label classes, and each example gets only one label. This is the default implementation.
    - multiclass: The case in which there can be more than two different label classes, but each example still gets only one label.
    - multilabel: The case in which there can be more than two different label classes, and each example can have more than one label.
"""

_KWARGS_DESCRIPTION = """
Args:
- references (array-like of shape (n_samples,) or (n_samples, n_classes)): Ground truth labels. Expects different input based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples,)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- prediction_scores (array-like of shape (n_samples,) or (n_samples, n_classes)): Model predictions. Expects different inputs based on use case:
    - binary: expects an array-like of shape (n_samples,)
    - multiclass: expects an array-like of shape (n_samples, n_classes)
    - multilabel: expects an array-like of shape (n_samples, n_classes)
- average (`str`): Type of average, and is ignored in the binary use case. Defaults to 'macro'. Options are:
    - `'micro'`: Calculates metrics globally by considering each element of the label indicator matrix as a label. Only works with the multilabel use case.
    - `'macro'`: Calculate metrics for each label, and find their unweighted mean.  This does not take label imbalance into account.
    - `'weighted'`: Calculate metrics for each label, and find their average, weighted by support (i.e. the number of true instances for each label).
    - `'samples'`: Calculate metrics for each instance, and find their average. Only works with the multilabel use case.
    - `None`:  No average is calculated, and scores for each class are returned. Only works with the multilabels use case.
- sample_weight (array-like of shape (n_samples,)): Sample weights. Defaults to None.
- max_fpr (`float`): If not None, the standardized partial AUC over the range [0, `max_fpr`] is returned. Must be greater than `0` and less than or equal to `1`. Defaults to `None`. Note: For the multiclass use case, `max_fpr` should be either `None` or `1.0` as ROC AUC partial computation is not currently supported for `multiclass`.
- multi_class (`str`): Only used for multiclass targets, where it is required. Determines the type of configuration to use. Options are:
    - `'ovr'`: Stands for One-vs-rest. Computes the AUC of each class against the rest. This treats the multiclass case in the same way as the multilabel case. Sensitive to class imbalance even when `average == 'macro'`, because class imbalance affects the composition of each of the 'rest' groupings.
    - `'ovo'`: Stands for One-vs-one. Computes the average AUC of all possible pairwise combinations of classes. Insensitive to class imbalance when `average == 'macro'`.
- labels (array-like of shape (n_classes,)): Only used for multiclass targets. List of labels that index the classes in
    `prediction_scores`. If `None`, the numerical or lexicographical order of the labels in
    `prediction_scores` is used. Defaults to `None`.
Returns:
    roc_auc (`float` or array-like of shape (n_classes,)): Returns array if in multilabel use case and `average='None'`. Otherwise, returns `float`.
Examples:
    Example 1:
        >>> roc_auc_score = evaluate.load("roc_auc")
        >>> refs = [1, 0, 1, 1, 0, 0]
        >>> pred_scores = [0.5, 0.2, 0.99, 0.3, 0.1, 0.7]
        >>> results = roc_auc_score.compute(references=refs, prediction_scores=pred_scores)
        >>> print(round(results['roc_auc'], 2))
        0.78
    Example 2:
        >>> roc_auc_score = evaluate.load("roc_auc", "multiclass")
        >>> refs = [1, 0, 1, 2, 2, 0]
        >>> pred_scores = [[0.3, 0.5, 0.2],
        ...                 [0.7, 0.2, 0.1],
        ...                 [0.005, 0.99, 0.005],
        ...                 [0.2, 0.3, 0.5],
        ...                 [0.1, 0.1, 0.8],
        ...                 [0.1, 0.7, 0.2]]
        >>> results = roc_auc_score.compute(references=refs, prediction_scores=pred_scores, multi_class='ovr')
        >>> print(round(results['roc_auc'], 2))
        0.85
    Example 3:
        >>> roc_auc_score = evaluate.load("roc_auc", "multilabel")
        >>> refs = [[1, 1, 0],
        ...         [1, 1, 0],
        ...         [0, 1, 0],
        ...         [0, 0, 1],
        ...         [0, 1, 1],
        ...         [1, 0, 1]]
        >>> pred_scores = [[0.3, 0.5, 0.2],
        ...                 [0.7, 0.2, 0.1],
        ...                 [0.005, 0.99, 0.005],
        ...                 [0.2, 0.3, 0.5],
        ...                 [0.1, 0.1, 0.8],
        ...                 [0.1, 0.7, 0.2]]
        >>> results = roc_auc_score.compute(references=refs, prediction_scores=pred_scores, average=None)
        >>> print([round(res, 2) for res in results['roc_auc']])
        [0.83, 0.38, 0.94]
"""

_CITATION = """\
@article{doi:10.1177/0272989X8900900307,
author = {Donna Katzman McClish},
title ={Analyzing a Portion of the ROC Curve},
journal = {Medical Decision Making},
volume = {9},
number = {3},
pages = {190-195},
year = {1989},
doi = {10.1177/0272989X8900900307},
    note ={PMID: 2668680},
URL = {https://doi.org/10.1177/0272989X8900900307},
eprint = {https://doi.org/10.1177/0272989X8900900307}
}
@article{10.1023/A:1010920819831,
author = {Hand, David J. and Till, Robert J.},
title = {A Simple Generalisation of the Area Under the ROC Curve for Multiple Class Classification Problems},
year = {2001},
issue_date = {November 2001},
publisher = {Kluwer Academic Publishers},
address = {USA},
volume = {45},
number = {2},
issn = {0885-6125},
url = {https://doi.org/10.1023/A:1010920819831},
doi = {10.1023/A:1010920819831},
journal = {Mach. Learn.},
month = {oct},
pages = {171–186},
numpages = {16},
keywords = {Gini index, AUC, error rate, ROC curve, receiver operating characteristic}
}
@article{scikit-learn,
title={Scikit-learn: Machine Learning in {P}ython},
author={Pedregosa, F. and Varoquaux, G. and Gramfort, A. and Michel, V. and Thirion, B. and Grisel, O. and Blondel, M. and Prettenhofer, P. and Weiss, R. and Dubourg, V. and Vanderplas, J. and Passos, A. and Cournapeau, D. and Brucher, M. and Perrot, M. and Duchesnay, E.},
journal={Journal of Machine Learning Research},
volume={12},
pages={2825--2830},
year={2011}
}
"""
@evaluate.utils.file_utils.add_start_docstrings(_DESCRIPTION, _KWARGS_DESCRIPTION)
class ROCAUC(evaluate.Metric):
    def _info(self):
        return evaluate.MetricInfo(
            description=_DESCRIPTION,
            citation=_CITATION,
            inputs_description=_KWARGS_DESCRIPTION,
            features=datasets.Features(
                {
                    "prediction_scores": datasets.Sequence(datasets.Value("float")),
                    "references": datasets.Value("int32"),
                }
                if self.config_name == "multiclass"
                else {
                    "references": datasets.Sequence(datasets.Value("int32")),
                    "prediction_scores": datasets.Sequence(datasets.Value("float")),
                }
                if self.config_name == "multilabel"
                else {
                    "references": datasets.Value("int32"),
                    "prediction_scores": datasets.Value("float"),
                }
            ),
            reference_urls=["https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html"],
        )

    def _compute(
        self,
        references,
        prediction_scores,
        average="macro",
        sample_weight=None,
        max_fpr=None,
        multi_class="raise",
        labels=None,
    ):
        return {
            "roc_auc": roc_auc_score(
                references,
                prediction_scores,
                average=average,
                sample_weight=sample_weight,
                max_fpr=max_fpr,
                multi_class=multi_class,
                labels=labels,
            )
        }

In [2]:
!wget https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/train.csv
!wget https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/test.csv


--2026-01-19 13:13:46--  https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1054962 (1.0M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.01M  --.-KB/s    in 0.05s   

2026-01-19 13:13:46 (20.9 MB/s) - ‘train.csv’ saved [1054962/1054962]

--2026-01-19 13:13:46--  https://raw.githubusercontent.com/tasinfrancesco/pml_challenge/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3243869 (3.1M) [text/plain]
Sav

In [ ]:
### long version of the same code ###

tokenizer = AutoTokenizer.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", padding = True)
model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/BERTOverflow_stackoverflow_github", num_labels = 2, problem_type = "single_label_classification")
batch_size = 16
num_epochs = 3
optimizer = AdamW(model.parameters(), lr = 5e-5, weight_decay = .01)

data_files = {"train": "train.csv", "test": "test.csv"}
loaded_ds = load_dataset("csv", data_files = data_files)
print(f"shape of 'tokenized_train': {loaded_ds["train"].shape}")
print(f"shape of 'tokenized_test': {loaded_ds["test"].shape}")

small_train_val_ds = DatasetDict(
    train = loaded_ds["train"].shuffle(seed=42).select(range(128)),
    val = loaded_ds["train"].shuffle(seed=42).select(range(128, 160)),
)


small_tokenized_ds = small_train_val_ds.map(lambda example : tokenizer(example["text"], padding = True),
                                            batched = True,
                                            batch_size = batch_size,
                                            )


small_tokenized_ds = small_tokenized_ds.remove_columns(["text"])
small_tokenized_ds = small_tokenized_ds.rename_column("target", "labels")
small_tokenized_ds.set_format("torch")

train_dataloader = DataLoader(small_tokenized_ds["train"], batch_size = batch_size)
eval_dataloader = DataLoader(small_tokenized_ds["val"], batch_size = batch_size)


num_training_steps = 3*len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps = num_training_steps)

best_val_loss = float("inf")
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
  model.train()
  for batch_i, batch in enumerate(train_dataloader):
    output = model(input_ids = batch["input_ids"],
                     attention_mask = batch["attention_mask"],
                     token_type_ids = batch["token_type_ids"],
                     labels = batch["labels"]
                   )
    optimizer.zero_grad()
    output.loss.backward()
    optimizer.step()
    lr_scheduler.step()
    progress_bar.update(1)
  model.eval()
  loss = 0
  for batch_i, batch in enumerate(eval_dataloader):
    with torch.no_grad():
      output = model(input_ids = batch["input_ids"],
                     attention_mask = batch["attention_mask"],
                     token_type_ids = batch["token_type_ids"],
                     labels = batch["labels"]
                     )
    loss += output.loss
  avg_val_loss = loss/len(eval_dataloader)
  print(f"\naverage validation loss = {avg_val_loss}")
  if avg_val_loss<best_val_loss:
    torch.save({
        "epoch" : epoch,
        "model_state_dict" : model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_loss": best_val_loss},
               f"epoch_{epoch}.pt"
               )


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at lanwuwei/BERTOverflow_stackoverflow_github and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


shape of 'tokenized_train': (5000, 7)
shape of 'tokenized_test': (15000, 7)


  0%|          | 0/24 [00:00<?, ?it/s]


average validation loss = 0.40866944193840027


In [37]:
help(model.forward(train_dataloader))

TypeError: 'DataLoader' object is not subscriptable